In [0]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [0]:
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Flatten
from keras.optimizers import Adam
from keras import initializers
from keras.utils import plot_model, np_utils
from keras import backend as K
from SpectralNormalizationKeras import DenseSN, ConvSN2D, ConvSN2DTranspose

Using TensorFlow backend.


In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
num_classes = len(np.unique(y_train))
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 3, 32, 32)
    X_test = X_test.reshape(X_test.shape[0], 3, 32, 32)
    input_shape = (3, 32, 32)
else:
    X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
    X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
    input_shape = (32, 32, 3)
    
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

# the generator is using tanh activation, for which we need to preprocess 
# the image data into the range between -1 and 1.

X_train = np.float32(X_train)
X_train = (X_train / 255 - 0.5) * 2
X_train = np.clip(X_train, -1, 1)

X_test = np.float32(X_test)
X_test = (X_test / 255 - 0.5) * 2
X_test = np.clip(X_test, -1, 1)

print('X_train reshape:', X_train.shape)
print('X_test reshape:', X_test.shape)

X_train reshape: (50000, 32, 32, 3)
X_test reshape: (50000, 32, 32, 3)


In [0]:
img_shape = X_train[0].shape
init = initializers.RandomNormal(stddev=0.02)

In [0]:
def get_generator(latent_dim, init):
    model = Sequential([
        Dense(2*2*512, input_shape=(latent_dim,), kernel_initializer=init),
        Reshape((2, 2, 512)),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh')
    ])
    print("Generator Model Summary")
    print(model.summary())
    return model

In [0]:
def get_generator_SN(latent_dim, init):
    model = Sequential([
        DenseSN(2*2*512, input_shape=(latent_dim,), kernel_initializer=init),
        Reshape((2, 2, 512)),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(64, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh')
    ])
    print("Generator Model Summary")
    print(model.summary())
    return model

In [0]:
latent_dim = 100

In [0]:
def get_discriminator(img_shape, init):
    model = Sequential([
        Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(img_shape), kernel_initializer=init),
        LeakyReLU(0.2),
        Conv2D(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2D(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2D(512, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    print("Discriminator Model Summary")
    print(model.summary())
    return model

In [0]:
def get_discriminator_SN(img_shape, init):
    model = Sequential([
        ConvSN2D(64, kernel_size=5, strides=2, padding='same', input_shape=(img_shape), kernel_initializer=init),
        LeakyReLU(0.2),
        ConvSN2D(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2D(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2D(512, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Flatten(),
        DenseSN(1, activation='sigmoid')
    ])
    print("Discriminator Model Summary")
    print(model.summary())
    return model

In [0]:
pip install tensorflow-gan

     |████████████████████████████████| 368kB 6.7MB/s 


In [0]:
import tensorflow_gan as tfgan

In [0]:
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    fid = tfgan.eval.frechet_classifier_distance_from_activations(act1, act2)
    return fid

In [0]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

In [0]:
import numpy
from numpy import cov

In [0]:
DC_generator = get_generator_SN(latent_dim, init)
DC_discriminator = get_discriminator_SN(img_shape, init)
DC_discriminator.compile(Adam(lr=0.0003, beta_1=0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])

Generator Model Summary
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_sn_1 (DenseSN)         (None, 2048)              208896    
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 512)         2048      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 2, 2, 512)         0         
_________________________________________________________________
conv_s_n2d_transpose_1 (Conv (None, 4, 4, 256)         3277568   
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (

In [0]:
def get_gan_model(generator, discriminator):
    discriminator.trainable = False
    z = Input(shape=(latent_dim,))
    img = generator(z)
    decision = discriminator(img)
    model = Model(inputs=z, outputs=decision)
    model.compile(Adam(lr=0.0004, beta_1=0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])
    print("GAN Model Summary")
    print(model.summary())
    return model

In [0]:
dcgan_model = get_gan_model(DC_generator, DC_discriminator)

GAN Model Summary
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 32, 32, 3)         4519747   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 4313154   
Total params: 8,832,901
Trainable params: 4,514,819
Non-trainable params: 4,318,082
_________________________________________________________________
None


In [0]:
def gan_train(gan_model, generator, discriminator, epochs, batch_size, smooth, show_train):
    real = np.ones(shape=(batch_size, 1))
    fake = np.zeros(shape=(batch_size, 1))

    d_loss = []
    g_loss = []
    for e in range(epochs + 1):
        for i in range(len(X_train) // batch_size):
            
            # Train Discriminator weights
            discriminator.trainable = True
            
            # Real samples
            X_batch = X_train[i*batch_size:(i+1)*batch_size]
            d_loss_real = discriminator.train_on_batch(x=X_batch, y=real * (1 - smooth))
            
            # Fake Samples
            z = np.random.normal(loc=0, scale=1, size=(batch_size, latent_dim))
            X_fake = generator.predict_on_batch(z)
            d_loss_fake = discriminator.train_on_batch(x=X_fake, y=fake)
            
            # Wasserstein loss
            d_loss_batch = tf.reduce_mean(d_loss_fake) - tf.reduce_mean(d_loss_real)
            
            # Train Generator weights
            discriminator.trainable = False
            g_loss_batch = gan_model.train_on_batch(x=z, y=real)

            print(
                'epoch = %d/%d, batch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, i, len(X_train) // batch_size, d_loss_batch, g_loss_batch[0]),
                100*' ',
                end='\r'
            )
        
        d_loss.append(d_loss_batch)
        g_loss.append(g_loss_batch[0])
        print('epoch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, d_loss[-1], g_loss[-1]), 100*' ')
        if show_train == True:
            #if e % 10 == 0:
            samples = 10
            x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples, latent_dim)))

            for k in range(samples):
                plt.subplot(2, 5, k + 1, xticks=[], yticks=[])
                plt.imshow(((x_fake[k] + 1)* 127).astype(np.uint8))

            plt.tight_layout()
            plt.show()
    return d_loss, g_loss

In [0]:
def self_attention( x, r = 8, reuse = False ) :
    [ _, h, w, c ] = x.get_shape().as_list()
    n = h * w
    cr = c // r
    pf = ConvSN2D( x, cr, 1, 1)
    pg = ConvSN2D( x, cr, 1, 1)
    ph = ConvSN2D( x, c, 1, 1)
    f = tf.reshape( pf, [ -1, n, cr ] ) 
    g = tf.reshape( tf.transpose( pg, [ 0, 3, 1, 2 ] ), [ -1, cr, n ] ) 
    h_ = tf.reshape( ph, [ -1, n, c ] ) 
    fg = tf.matmul( f, g ) # n by n matrix
    fg = tf.nn.softmax( fg, axis = 1 ) 
    att = tf.matmul( fg, h_ ) # n by c matrix
    att_fm = tf.reshape( att, [ -1, h, w, c ] ) 
    y = tf.get_variable( 'y', shape = [], dtype = tf.float32, initializer=tf.constant_initializer( 0 ) ) 
    return x + y * att_fm

In [0]:
#Hyperparameters
epochs = 1000
batch_size = 4096
smooth = 0.1

In [0]:
d_loss, g_loss = gan_train(dcgan_model, DC_generator, DC_discriminator, epochs, batch_size, smooth, False)

epoch = 1/1000, d_loss=0.311, g_loss=6.933                                                                                                     
epoch = 2/1000, d_loss=0.331, g_loss=0.004                                                                                                     
epoch = 3/1000, d_loss=0.329, g_loss=0.002                                                                                                     
epoch = 4/1000, d_loss=0.329, g_loss=0.001                                                                                                     
epoch = 5/1000, d_loss=0.332, g_loss=0.001                                                                                                     
epoch = 6/1000, d_loss=0.332, g_loss=0.001                                                                                                     
epoch = 7/1000, d_loss=0.334, g_loss=0.001                                                                                              

In [0]:
DC_generator.save('./DC_generator_model.h5')

In [0]:
DC_discriminator.save('./DC_discriminator_model.h5')

In [0]:
discriminator.predict(None, X_test[0])

In [0]:
len(X_test)

In [0]:
x_fake = DC_generator.predict(np.random.normal(loc=0, scale=1, size=(1, latent_dim)))

In [0]:
for k in range(1):
    plt.subplot(2, 5, k + 1, xticks=[], yticks=[])
    plt.imshow(((x_fake[k] + 1)* 127).astype(np.uint8))

plt.tight_layout()
plt.show()

In [0]:
x1 = []
x1.append(X_test[0])

In [0]:
fid1 = calculate_fid(DC_discriminator, x_fake, x1)
print(fid1)

In [0]:
x_fake = DC_generator.predict(np.random.normal(loc=0, scale=1, size=(1, latent_dim)))
#x_fake = np.asarray(x_fake)
#x1 = np.asarray(x1)
fid1 = calculate_fid(DC_discriminator, x_fake, x1)
print(fid1)

In [0]:
calculate_fid(dcgan_model, )

In [0]:
!pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

In [0]:
def get_generator_SN_SA(latent_dim, init):
    model = Sequential([
        DenseSN(2*2*512, input_shape=(latent_dim,), kernel_initializer=init),
        Reshape((2, 2, 512)),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2DTranspose(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        AttentionLayer((None, 8, 8, 128)),
        ConvSN2DTranspose(64, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        AttentionLayer((None, 32, 32, 3)),
        ConvSN2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh')
    ])
    print("Generator Model Summary")
    print(model.summary())
    return model

In [0]:
def get_discriminator_SN_SA(img_shape, init):
    model = Sequential([
        ConvSN2D(64, kernel_size=5, strides=2, padding='same', input_shape=(img_shape), kernel_initializer=init),
        LeakyReLU(0.2),
        ConvSN2D(128, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        ConvSN2D(256, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        SeqSelfAttention(attention_activation='sigmoid'),
        ConvSN2D(512, kernel_size=5, strides=2, padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        SeqSelfAttention(attention_activation='sigmoid'),
        Flatten(),
        DenseSN(1, activation='sigmoid')
    ])
    print("Discriminator Model Summary")
    print(model.summary())
    return model

In [0]:
SA_generator = get_generator_SN_SA(latent_dim, init)
SA_discriminator = get_discriminator_SN_SA(img_shape, init)
SA_discriminator.compile(Adam(lr=0.0003, beta_1=0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [0]:
sagan_model = get_gan_model(SA_generator, SA_discriminator)

In [0]:
gan_train(sagan_model, SA_generator, SA_discriminator, epochs, batch_size, smooth, True)

In [0]:
plt.plot(d_loss)
plt.plot(g_loss)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Discriminator', 'Adversarial'], loc='center right')
plt.show()